----
-----
-----

## **Mean = 0, Std = 1 olacak şekilde yeniden ölçeklemek**

**LayerNorm, mean ve std’yi minimize eden bir yapı değil; her forward adımında mean=0, std=1 olacak şekilde aktivasyonları yeniden ölçekleyen deterministik bir normalizasyon katmanıdır.**

LN:

* her örneği kendi içinde normalize eder

* batch gürültüsünü yok sayar

* gradyan akışını stabilize eder

Ama:

* “mean’i sıfıra çekmeye çalışıyor” diye bir hedef yok

* her forward pass’te zorla sıfır yapıyor

-----
-----
-----

# 4) Normalization Katmanları — Layer Normalization (LayerNorm)

Bu defter, **Layer Normalization (LN)** kavramını **sıfırdan ileri seviyeye** taşır:
- LN nedir, hangi probleme cevap verir?
- Matematiksel tanım (forward) ve pratik nüanslar
- CNN (object detection / segmentation) bağlamında nerede mantıklı?
- PyTorch implementasyonları: `nn.LayerNorm`, **channels-first (NCHW)** için pratik çözümler
- BN/GN/IN ile karşılaştırma (kısa ama net)

> Not: Bu defter *uygulama odaklıdır*. Aşırı teori yerine, doğru sezgi + doğru kod + doğru kullanım senaryosu hedeflenir.


---

## 0) Normalization kavramı: Neyi normalize ediyoruz?

Bir sinir ağında "normalization" genel olarak şu iki şeyi hedefler:

1. **Optimizasyonu stabilize etmek**  
   Aktivasyonların ölçeği/dağılımı aşırı oynadığında, gradyanlar patlayabilir/sönebilir ve eğitim zorlaşır.

2. **Hiperparametre hassasiyetini azaltmak**  
   Öğrenme oranı (LR), başlangıç ağırlıkları, batch boyutu gibi değişkenlere toleransı artırır.

Normalization ailesi, bunu farklı "eksenlerde" yapar:
- Batch üzerinde mi? (BN)
- Kanal grupları üzerinde mi? (GN)
- Örnek başına mı? (IN)
- Katman başına mı? (LN)

LayerNorm bu ailenin önemli bir üyesidir çünkü **batch boyutuna bağımlılığı yoktur**.


---

## 1) Layer Normalization (LN) nedir?

**Layer Normalization**, bir örnek (sample) içinde, belirli bir katmandaki aktivasyonları normalize eder.

En basit haliyle:
- Her örnek için ayrı ayrı,
- Belirli boyut seti üzerinde (genelde feature dimension),
- Ortalama (`μ`) ve standart sapma (`σ`) hesaplanır,
- Aktivasyonlar normalize edilir,
- Ardından **öğrenilebilir** ölçek (`γ`) ve kaydırma (`β`) uygulanır.

Bu sayede ağ, "normalize edilmiş" uzayda stabil şekilde öğrenirken,
gerekli olduğunda `γ` ve `β` ile tekrar uygun ölçeğe dönebilecek esnekliği korur.


### 1.1) Nereden çıkmıştır? Neden ihtiyaç doğdu?

Batch Normalization (BN) CNN'lerde çok güçlü olsa da şu iki ana problem öne çıkar:

1. **Batch bağımlılığı**  
   BN istatistikleri batch üzerinden hesaplar. Batch küçükse (özellikle detection/segmentation), istatistikler gürültülü olur.

2. **Training / inference davranışı ayrımı**  
   BN'de eğitimde batch istatistiği, inference'ta running mean/var kullanılır.
   Dağılım kayarsa veya fine-tune senaryolarında bu karışıklık sorun yaratabilir.

LayerNorm, özellikle **RNN/Transformer gibi** sequence modellerinde batch boyutu değişken ve küçük olabildiği için,
**batch'e bağımlı olmadan** stabilizasyon sağlamak amacıyla geliştirilmiştir.
Daha sonra CNN dünyasında da (özellikle small-batch rejimi) anlamlı kullanım alanları bulmuştur.


---

## 2) Matematik (temel): LN forward denklemi

Bir örnek için normalize edilen boyutları `S` ile gösterelim (ör: embedding dimension).

- Ortalama:
\[
\mu = \frac{1}{|S|}\sum_{i\in S} x_i
\]

- Varyans:
\[
\sigma^2 = \frac{1}{|S|}\sum_{i\in S} (x_i-\mu)^2
\]

- Normalize:
\[
\hat{x}_i = \frac{x_i-\mu}{\sqrt{\sigma^2 + \epsilon}}
\]

- Öğrenilebilir ölçek/kaydırma:
\[
y_i = \gamma_i\hat{x}_i + \beta_i
\]

Burada:
- `ε` sayısal stabilite içindir (sıfıra bölmeyi önler),
- `γ` ve `β` parametreleri şekil olarak normalize edilen boyutlarla uyumludur.


### 2.1) LN hangi eksenlerde çalışır?

Bu kritik bir noktadır: LN "layer" kelimesi yüzünden bazen yanlış anlaşılır.

- **MLP / Transformer**: genelde `(..., d_model)` üzerinde normalize edilir → yani **son boyut**.
- **CNN**: NCHW formatında (batch, channel, height, width) tipik LN kullanımları:
  - **channels-last** (NHWC) + `LayerNorm(C)` (çok yaygın)
  - NCHW için özel bir katman ile **kanal ekseninde** normalize etme (ConvNeXt tarzı)

PyTorch `nn.LayerNorm` varsayılan olarak **son boyutları** normalize eder.
Bu yüzden CNN'de NCHW ile doğrudan kullanmak istediğinde genelde iki seçenek vardır:
1. NCHW → NHWC (channels-last) formatına geçip `LayerNorm(C)` uygulamak
2. NCHW için özel `LayerNorm2d` yazarak C ekseni üzerinde normalize etmek


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

---

## 3) PyTorch ile LN: en temel kullanım

`nn.LayerNorm(normalized_shape)`:

- `normalized_shape`: normalize edilecek son boyut(lar)ın şekli
  - Örn: `[d_model]` veya `d_model`
  - Örn: `[C, H, W]` gibi çoklu boyutlar da verilebilir (tensor düzeni uygunsa)

Önemli: `LayerNorm` **training / inference** arasında aynı davranır.
BN'deki gibi running stats ayrımı yoktur.


In [ ]:
x = torch.randn(2, 5)          # (batch=2, features=5)
ln = nn.LayerNorm(5)

y = ln(x)
print("x mean/std (sample0):", x[0].mean().item(), x[0].std(unbiased=False).item())
print("y mean/std (sample0):", y[0].mean().item(), y[0].std(unbiased=False).item())

x mean/std (sample0): -0.03851224109530449 0.546144425868988
y mean/std (sample0): 0.0 0.9999833106994629


Beklenen:
- Her örnek için (satır başına) ortalama ~0, std ~1 (γ=1, β=0 iken).


---

## 4) CNN tarafı: NCHW ile LN nasıl yapılır?

NCHW: `(N, C, H, W)`

`nn.LayerNorm` ile `(C,H,W)` üzerinde normalize etmek mümkündür ama:
- `H,W` sabit olmalı (input boyutu değişirse shape tutmaz),
- Detection/segmentation gibi dinamik boyutlarda bu pratik değildir.

Bu yüzden CNN'de iki yaygın pratik yaklaşım:
1) **Channels-last** (NHWC) kullanıp `LayerNorm(C)` yapmak  
2) **C ekseninde** normalize eden bir `LayerNorm2d` yazmak (ConvNeXt benzeri)

Aşağıdaki implementasyon NCHW tensor üzerinde kanal ekseninde LN yapar:
- Her piksel konumu için C üzerinden normalize eder (per-position).


In [3]:
class LayerNorm2d(nn.Module):
    def __init__(self, num_channels: int, eps: float = 1e-5, affine: bool = True):
        super().__init__()
        self.eps = eps
        self.affine = affine
        if affine:
            self.gamma = nn.Parameter(torch.ones(num_channels))
            self.beta = nn.Parameter(torch.zeros(num_channels))
        else:
            self.register_parameter('gamma', None)
            self.register_parameter('beta', None)

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)  # over C
        var = (x - mean).pow(2).mean(dim=1, keepdim=True)
        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        if self.affine:
            return x_hat * self.gamma.view(1, -1, 1, 1) + self.beta.view(1, -1, 1, 1)
        return x_hat

# hızlı kontrol
x_img = torch.randn(2, 3, 4, 4)
ln2d = LayerNorm2d(3)
y_img = ln2d(x_img)
print("Per-position mean (sample0, h0,w0):", y_img[0, :, 0, 0].mean().item())
print("Per-position std  (sample0, h0,w0):", y_img[0, :, 0, 0].std(unbiased=False).item())

Per-position mean (sample0, h0,w0): 0.0
Per-position std  (sample0, h0,w0): 0.9999761581420898


---

## 5) LN vs BN: detection/segmentation için kritik farklar

### Batch bağımlılığı
- BN: batch istatistikleri → batch küçükse gürültü
- LN: örnek içi istatistik → batch'ten bağımsız

### Training/Inference davranışı
- BN: train/infer farklı (running stats)
- LN: aynı (deterministik)

### CNN performansı (pratik gözlem)
- Klasik classification CNN (büyük batch): BN çoğu zaman daha iyi/kolay
- Detection/segmentation (small batch): GN/LN/IN gibi batch bağımsız normlar avantajlı olabilir


---

## 6) Mini deney: Küçük batch'te BN vs LN davranışı (hızlı)

Bu deney:
- Küçük batch rejiminde BN ve LN-benzeri yaklaşımın farklı davranabileceğini gösterir.
- Gerçek dataset yerine `torchvision.datasets.FakeData` ile hızlı demo yapılır.
- Bu bir benchmark değildir; sezgi kazandırma amaçlıdır.


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])
train_ds = datasets.FakeData(size=512, image_size=(3, 32, 32), num_classes=10, transform=transform)

def make_loader(batch_size):
    return DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)

class SmallCNN_BN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(64, 10)
    def forward(self, x):
        x = self.net(x).flatten(1)
        return self.fc(x)

class SmallCNN_LN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            LayerNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            LayerNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(64, 10)
    def forward(self, x):
        x = self.net(x).flatten(1)
        return self.fc(x)

def one_epoch(model, loader, lr=1e-2, device="cpu"):
    model.to(device)
    model.train()
    opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    total_loss = 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        opt.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

for bs in [2, 16]:
    loader = make_loader(bs)
    bn_model = SmallCNN_BN()
    ln_model = SmallCNN_LN()
    bn_loss = one_epoch(bn_model, loader)
    ln_loss = one_epoch(ln_model, loader)
    print(f"Batch size={bs:>2} | BN loss={bn_loss:.4f} | LN-like loss={ln_loss:.4f}")


Batch size= 2 | BN loss=2.4088 | LN-like loss=2.3993
Batch size=16 | BN loss=2.3126 | LN-like loss=2.3742


---

## 7) CNN'de LayerNorm ne zaman mantıklı?

### Mantıklı olduğu yerler
- **Batch size küçük** (detection/segmentation eğitimi)
- SyncBN istemiyorsun veya distributed senaryo yok
- Mimari zaten **channels-last** optimizasyonuna uygunsa
- Backbone'da BN yerine batch bağımsız norm denemek istiyorsun

### Dikkat gerektiren yerler
- Klasik classification CNN (büyük batch) → BN çoğu zaman daha iyi/kolay
- NCHW ile LN'yi yanlış eksende uygulamak → performans/kararlılık kaybı


---

## 8) LN için pratik “doğru kullanım” checklist'i ✅

1) **Hangi eksenlerde normalize ettiğini net yaz**  
2) **Affine (γ,β) açık mı?** (genelde açık)  
3) **eps** sayısal stabilite (karma hassasiyette bazen kritik)  
4) CNN formatı: **NHWC mi NCHW mi?**  
5) Detection/segmentation için gerçek metriklerle ölç: **mAP / IoU / Dice**


---

## 9) Deneme/ödev fikirleri (ileri seviye)

1) Aynı backbone üzerinde BN → GN → LN değiştir ve küçük batch (2/4) rejiminde mAP/IoU farkını kıyasla.  
2) LN'yi sadece neck'te kullan (FPN/PAN) ve etkisini ölç.  
3) `LayerNorm2d` yerine `channels-last + nn.LayerNorm(C)` ile hız/VRAM kıyasla.  
4) Mixed precision (fp16) eğitimde eps değerini değiştirip stabiliteyi gözle.


---

## 10) Özet

- **LayerNorm**, batch'e bağımlı olmadan normalize eder.
- **Training / inference** davranışı aynıdır (BN gibi running stats yok).
- CNN'de kullanırken **eksen seçimi** kritik:
  - Transformer/MLP: son boyut
  - CNN: channels-last LN(C) veya NCHW için LN-benzeri katman (ör. `LayerNorm2d`)
- Detection/segmentation gibi **small-batch** senaryolarda LN/GN ailesi sık tercih edilir.

Bir sonraki defter: **Group Normalization (GN)** — LN ile farkı ve small-batch detection/segmentation’da neden çok popüler olduğu.
